# Условие задачи

Главные задачи, которые должен выполнять скрипт:

- Извлекать реплики с приветствием – где менеджер поздоровался. 
- Извлекать реплики, где менеджер представил себя. 
- Извлекать имя менеджера. 
- Извлекать название компании. 
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


# Загрузка данных, импортирование библиотек

Необходимо подготовить данные, для того чтобы их можно было использовать для обучения.

In [1]:
#!pip install nltk

In [2]:
pip install Natasha

In [3]:
pip install navec

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import matplotlib as plt
import numpy as np
import math

#import warnings
#warnings.filterwarnings("ignore")

In [5]:
#import nltk
#from nltk.corpus import stopwords as nltk_stopwords
#from nltk.tokenize import word_tokenize 
#from nltk.stem import WordNetLemmatizer 
#from nltk.tokenize import RegexpTokenizer
#from nltk.probability import FreqDist

In [6]:
from yargy import Parser, rule, or_, and_
from yargy.interpretation import fact
from yargy.predicates import gram, eq, is_capitalized, length_eq 
from yargy.pipelines import morph_pipeline
from slovnet import NER
from navec import Navec

In [7]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    PER,
    LOC,
    NamesExtractor,
    MorphVocab,
    Doc
)

In [8]:
#segmenter = Segmenter()

#emb = NewsEmbedding()
#morph_tagger = NewsMorphTagger(emb)
#syntax_parser = NewsSyntaxParser(emb)
#morph_vocab  =  MorphVocab()
#names_extractor = NamesExtractor(morph_vocab)

In [9]:
#data = pd.read_csv('https://drive.google.com/file/d/1boACz8ab9UytCpi0QhTUFRyS2oyH1MXS/view?usp=sharing',encoding='utf-8')

In [10]:
data = pd.read_csv('test_data.csv',encoding='utf-8')

In [11]:
data.head(5)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


Загружены и импортированы библиотеки для работы с текстом **natasha** и **nltk**. В процессе работы можно будет воспользоваться той что удобнее. Данные скачаны и проверены на корректность.

# Предварительный анализ данных и проверка рабочих инструментов

Необходимо провести предварительный анализ данных, и подготовится к выполнению задачи.

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 11.3+ KB


In [13]:
data.loc[data['line_n'] == 1,:]

,dlg_id,line_n,role,text
1,0,1,manager,Алло здравствуйте
110,1,1,manager,Алло здравствуйте
165,2,1,client,Здравствуйте
250,3,1,manager,Алло дмитрий добрый день
303,4,1,client,Алло
338,5,1,manager,Да это анастасия


Данные 

In [14]:
data_manager = data[data['role'] == 'manager']

In [15]:
data_manager.loc[3, "text"]

'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается'

Информация выглядит упорядоченной. Судя по столбцу **dlg_id** у нас шесть диалогов, в каждом из которых есть разделение на отдельные реплики с подписью от кого они исходят. Каждая реплика начинается с заглавной буквы но остальные буквы строчные, в том числе миена и названия компании. Знаков препинания нет. Приветствие и представление может быть как в первой реплике так и в последующих. Реплики не имеют чёткой структуры и последовательности, например в начале могут идти как слова клиента так и менеджера, как в единственном числе так и несколько реплик подряд.

Для выполнения задачи нужно привести все слова к единому регистру, лемматизировать и извлечь нужную информацию. Для лемматизации и извлечения имён использую библиотеку **natasha**. 

В начале проверю перечисленные действия на небольших участках текста.

In [16]:
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
#syntax_parser = NewsSyntaxParser(emb)
morph_vocab  =  MorphVocab()
names_extractor = NamesExtractor(morph_vocab)
segmenter = Segmenter()

In [17]:
doc = Doc(data_manager.loc[3, "text"])
doc

Doc(text='Меня зовут ангелина компания диджитал бизнес звон...)

In [18]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [19]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)
    
#{_.text: _.lemma for _ in doc.tokens[:10]}.values()
[_.lemma for _ in doc.tokens]

['я',
 'звать',
 'ангелина',
 'компания',
 'диджитал',
 'бизнес',
 'звонить',
 'вы',
 'по',
 'повод',
 'продление',
 'лицензия',
 'а',
 'мы',
 'с',
 'серый',
 'у',
 'вы',
 'скоро',
 'срок',
 'заканчиваться']

In [20]:
#from natasha import NewsNERTagger
#ner_tagger = NewsNERTagger(emb)
#doc.tag_ner(ner_tagger)
#doc.ner.print()

In [21]:
z= ' '
for i in range(0, 10):
  z += str(data.loc[i, "text"]) + ' '

In [22]:
z

' Алло Алло здравствуйте Добрый день Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается Ага Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать Да мы работаем с компанией которая нам подливает поэтому спасибо огромное Как как бы уже до этого момента работаем все устраивает + у нас сопровождение поэтому Угу а на что вы обращаете внимание при выборе Как бы нет '

In [23]:
a = Doc(z.lower())
a.segment(segmenter)
a.tag_morph(morph_tagger)

In [24]:
for token in a.tokens:
     token.lemmatize(morph_vocab)

In [25]:
[_.lemma for _ in a.tokens[:20]]

['алло',
 'алло',
 'здравствовать',
 'добрый',
 'день',
 'я',
 'звать',
 'ангелина',
 'компания',
 'диджитал',
 'бизнес',
 'звонить',
 'вы',
 'по',
 'повод',
 'продление',
 'лицензия',
 'а',
 'мы',
 'с']

In [26]:
q = ' '.join([_.lemma for _ in a.tokens])

In [27]:
q

'алло алло здравствовать добрый день я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться ага угу ну возможно вы рассмотреть и другой вариант видеть это хороший практика сравнивать да мы работать с компания который мы подливать поэтому спасибо огромный как как бы уже до этот момент работать весь устраивать + у мы сопровождение поэтому угу а на что вы обращать внимание при выбор как бы нет'

In [28]:
names_extractor.find(q)

Match(
    start=10,
    stop=23,
    fact=Name(
        first=None,
        last='здравствовать',
        middle=None
    )
)

Можно убедиться что библиотека **natasha** действительно хорошо лемматизирует текст и выделяет имена, при этом даже когда они написаны со строчной буквы. Но с выделением приветствия и прощания могут возникнуть проблемы. Поэтому необходимо подготовить шаблоны для поиска со списком близких слов.

In [29]:
greeting_sing = {'привет', 'добрый', 'здравствовать'}

In [30]:
name = Doc('имя зовут представиться')
name.segment(segmenter)
name.tag_morph(morph_tagger)
for token in name.tokens:
     token.lemmatize(morph_vocab)
name_sing = set(_.lemma for _ in name.tokens)
name_sing 

{'звать', 'имя', 'представиться'}

In [31]:
company = Doc('компания фирма группа представляю называется')
company.segment(segmenter)
company.tag_morph(morph_tagger)
for token in company.tokens:
     token.lemmatize(morph_vocab)
company_sing = set(_.lemma for _ in company.tokens)
company_sing  

{'группа', 'компания', 'называться', 'представляю', 'фирма'}

In [32]:
farewell = Doc('свидания встречи созвона благ успехов пока')
farewell.segment(segmenter)
farewell.tag_morph(morph_tagger)
for token in farewell.tokens:
     token.lemmatize(morph_vocab)
farewell_sing = set(_.lemma for _ in farewell.tokens)
farewell_sing

{'благо', 'встреча', 'пока', 'свидание', 'созвон', 'успех'}

Шаблоны с признаками приветствия, представления, названия фирмы и прощания готовы.

# Решение задачи

Создам правило для извлечения компании

In [33]:
Orgname = fact(
                  'Org',
                  ['orgform', 'name'])

ORGFORM = morph_pipeline([
                       'компания ',
                       'фирма '])

NAME = or_(gram('Surn'),
                    gram('Name'),
                    gram('NOUN'))
ORGANIZATION = rule(ORGFORM.interpretation(Orgname.orgform), NAME.interpretation(Orgname.name).interpretation(Orgname))
orgparser = Parser(ORGANIZATION)

Необходимо написать ряд функций для каждой задачи.

In [34]:
def manager_control_greeting(data):
    """checking the greeting"""
    doc = Doc(data['text'])
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    lemmas = set(_.lemma for _ in doc.tokens)
    if lemmas & greeting_sing:
        return f'разговор номер: {data["dlg_id"]},  реплика номер: {data["line_n"]},  фраза с приветствием: {data["text"]}'   

In [35]:
def manager_control_farewell(data):
    """checking the goodbye"""
    doc = Doc(data['text'])
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    lemmas = set(_.lemma for _ in doc.tokens)
    if lemmas & farewell_sing:
        return f'разговор номер: {data["dlg_id"]},  реплика номер: {data["line_n"]},  фраза с прощанием: {data["text"]}'   

In [36]:
def manager_control_name(data):
    """checking the representation and extracting the name"""
    doc = Doc(data['text'])
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    lemmas = set(_.lemma for _ in doc.tokens)
    if lemmas & name_sing:
        name_manager = names_extractor.find(data["text"].lower())
        return f'разговор номер: {data["dlg_id"]},  реплика номер: {data["line_n"]},  имя менеджера: {name_manager}'

In [37]:
def manager_control_company(data):
    
    """
     
     checking the presentation and extracting the company name
     
    """
    doc = Doc(data['text'])
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    lemmas = set(_.lemma for _ in doc.tokens)
    if lemmas & company_sing:
        comp_name = ['компания', 'ошибка извлечения']
        for i in orgparser.findall(data['text']):
            comp_name = [token.value for token in i.tokens]
        return f'разговор номер: {data["dlg_id"]},  реплика номер: {data["line_n"]}, название компании: {comp_name[1]}'

In [38]:
def manager_control_data(data):
    """
    checking the requirements for the manager
    """
    doc = Doc(data['text'])
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    lemmas = set(_.lemma for _ in doc.tokens)
    if lemmas & greeting_sing:
        return data["dlg_id"], 'приветствие'
    if lemmas & farewell_sing:
        return data["dlg_id"], 'прощание'

In [39]:
def manager_control(data):
    w = list(filter(None, set(data[data['role'] =='manager'].apply(manager_control_data, axis = 1))))
    rez = []
    for x in w:
        for y in w:
            if x[0] == y[0] and x[1] != y[1]:
                rez.append(f'в разговоре {x[0]} условие приветствие-прощание соблюдено')
    return set(rez)

In [40]:
set(data[data['role'] =='manager'].apply(manager_control_greeting, axis = 1))

{None,
 'разговор номер: 0,  реплика номер: 1,  фраза с приветствием: Алло здравствуйте',
 'разговор номер: 1,  реплика номер: 1,  фраза с приветствием: Алло здравствуйте',
 'разговор номер: 2,  реплика номер: 2,  фраза с приветствием: Алло здравствуйте',
 'разговор номер: 3,  реплика номер: 1,  фраза с приветствием: Алло дмитрий добрый день',
 'разговор номер: 3,  реплика номер: 2,  фраза с приветствием: Добрый меня максим зовут компания китобизнес удобно говорить',
 'разговор номер: 3,  реплика номер: 51,  фраза с приветствием: Угу все хорошо да понедельника тогда всего доброго'}

In [41]:
set(data[data['role'] =='manager'].apply(manager_control_name, axis = 1))

{None,
 "разговор номер: 0,  реплика номер: 3,  имя менеджера: Match(start=11, stop=19, fact=Name(first='ангелина', last=None, middle=None))",
 "разговор номер: 1,  реплика номер: 2,  имя менеджера: Match(start=11, stop=19, fact=Name(first='ангелина', last=None, middle=None))",
 "разговор номер: 2,  реплика номер: 3,  имя менеджера: Match(start=11, stop=19, fact=Name(first='ангелина', last=None, middle=None))",
 "разговор номер: 3,  реплика номер: 2,  имя менеджера: Match(start=0, stop=6, fact=Name(first=None, last='добрый', middle=None))"}

In [42]:
set(data[data['role'] =='manager'].apply(manager_control_company, axis = 1))

{None,
 'разговор номер: 0,  реплика номер: 3, название компании: диджитал',
 'разговор номер: 1,  реплика номер: 2, название компании: диджитал',
 'разговор номер: 2,  реплика номер: 3, название компании: диджитал',
 'разговор номер: 3,  реплика номер: 19, название компании: ошибка извлечения',
 'разговор номер: 3,  реплика номер: 2, название компании: китобизнес',
 'разговор номер: 3,  реплика номер: 24, название компании: ошибка извлечения'}

In [43]:
set(data[data['role'] =='manager'].apply(manager_control_farewell, axis = 1))

{None,
 'разговор номер: 0,  реплика номер: 108,  фраза с прощанием: Всего хорошего до свидания',
 'разговор номер: 1,  реплика номер: 54,  фраза с прощанием: До свидания',
 'разговор номер: 2,  реплика номер: 22,  фраза с прощанием: Контроль например отказов то есть только для вас подойдет для этого вам наш аналитик уже подберет что для вас будет более актуально поэтому я хотела бы вам предложить встречу там на 15 20 минут',
 'разговор номер: 4,  реплика номер: 33,  фраза с прощанием: Во вторник все ну с вами да тогда до вторника до свидания',
 'разговор номер: 5,  реплика номер: 142,  фраза с прощанием: Ну до свидания хорошего вечера'}

In [44]:
manager_control(data)

{'в разговоре 0 условие приветствие-прощание соблюдено',
 'в разговоре 1 условие приветствие-прощание соблюдено',
 'в разговоре 2 условие приветствие-прощание соблюдено'}

# Результат и пояснения

Задачу работы с текстами можно решать разными методами. С помощью регулярных выражений, с помощью библиотеки nltk, с помощью нейронных сетей, с помощью библиотеки Natasha. Я использовал последний метод. Для определения приветствия и прощания я лемматизировал текст и сравнил каждую реплику со списком характерных для приветствия и прощания слов и выражений. Качество получил удовлетворительное но не выдающееся. Предполагаемые методы улучшения качества - составление сложных условий (например, для приветствия, на то что в фразе одновременно встречаются слова "добрый" и "день"). Для извлечения имени я использовал стандартный инструмент - **NamesExtractor**. Для извлечения названия компании я составил правило с помощью встроенного в библиотеку Natasha yargy-парсера. 
В результате получил ряд функций. Одну из них (проверка приветствия-прощания) я завернул другую функцию, чтобы повысить удобство вызова.